In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [2]:
monitor_root = "../../data/csv/monitor"
policy_root = "../../data/policy/"
input_root = "../../data/output/xgbooster/"

In [3]:
monitor_coords = pd.read_csv(os.path.join(monitor_root, "monitor_processed.csv"),
                             index_col='id')
monitor_coords = monitor_coords[['city_cn']]

In [4]:
policy = pd.read_csv(os.path.join(policy_root, "PM2.5_reporting_date.csv"),
                     index_col='city_cn')

In [5]:
pred = pd.DataFrame()
for target_name in ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3"]:
    df = pd.read_csv(os.path.join(input_root, target_name, "prediction_output.csv"),
                     index_col=['id', 'date'])
    df.columns = ['pred_' + target_name]
    if pred.empty:
        pred = df
    else:
        pred = pred.join(df, how='outer')

In [6]:
pred.reset_index(inplace=True)

In [10]:
pred['date'] = pd.to_datetime(pred['date'])
pred['year'] = pred['date'].dt.year
pred['month'] = pred['date'].dt.month
pred['week'] = pred['date'].dt.dayofyear // 7

In [15]:
pred = pred.groupby(['year', 'week', 'id']).agg({
    'pred_PM2.5': 'mean',
    'pred_PM10': 'mean',
    'pred_SO2': 'mean',
    'pred_NO2': 'mean',
    'pred_CO': 'mean',
    'pred_O3': 'mean',
    'month': 'median'
})

In [16]:
pred = pred.join(monitor_coords)

In [17]:
pred = pred.join(policy, on='city_cn')

In [18]:
pred

pred_PM10    pred_O3   pred_NO2  pred_PM2.5   pred_CO  \
year week id                                                              
2005 0    1001A   93.011356  28.608202  53.519597   63.725131  1.545789   
          1002A  100.247579  36.950301  45.941632   95.209747  1.789880   
          1003A   98.808754  29.325108  52.563054   75.921958  1.881769   
          1004A   89.367277  30.325090  56.883238   63.517094  1.544300   
          1005A  110.084605  31.851208  69.765810   97.002444  1.889709   
          1006A   90.303267  23.959218  59.433468   60.578987  1.479513   
          1007A  115.930890  18.711527  68.313146   83.160965  2.276866   
          1008A         NaN  33.594915  48.745627   91.114088  1.619710   
          1009A         NaN  31.012222  38.890380   79.014516  1.387865   
          1010A  123.840881  27.307504  65.695341   91.302496  1.989450   
          1011A         NaN  30.657370  56.591031   69.462457  1.719192   
          1012A  110.723873  33.318767  48.373912   69.289188  1.763421   
          1013A         NaN  30.220529  71.102398         NaN  2.350275   
          1014A  137.218398  21.879464  59.850533   85.029465  2.147361   
          1015A  133.419387  22.900598  58.491924   99.765116  2.061054   
          1016A         NaN        NaN  65.302811         NaN       NaN   
          1017A  146.876282  23.915302  63.978990   78.419538  2.106106   
          1018A  146.296511  22.187275  72.029877   92.038177  1.812256   
          1019A  133.944970  23.889385  56.103978   87.374650  1.726805   
          1020A         NaN  24.367127        NaN         NaN  1.870890   
          1021A  132.351814  24.522600  65.326536   92.938363  2.115723   
          1023A         NaN  26.269937        NaN         NaN  1.789710   
          1024A  116.522294  20.112207  53.588872   70.783820       NaN   
          1025A  129.973885  27.154371  53.983339   85.605972  2.028240   
          1026A  114.332540  28.398884  48.298452   72.110037       NaN   
          1027A  126.515814  21.474943  64.341901  100.872447  1.922036   
          1028A         NaN        NaN        NaN         NaN  3.183762   
          1029A  210.787135        NaN  79.741716  164.795046  3.128179   
          1030A         NaN        NaN  83.803633         NaN  2.808819   
          1031A         NaN  19.575982  63.913755         NaN  2.839635   
...                     ...        ...        ...         ...       ...   
2016 52   2683A         NaN  23.993664        NaN         NaN  1.403759   
          2684A         NaN        NaN  38.966925         NaN  1.730310   
          2685A         NaN        NaN        NaN         NaN  1.334089   
          2686A         NaN        NaN  53.989479  116.917658  3.150743   
          2687A         NaN  14.840738  52.554661  126.534871  3.532384   
          2688A         NaN        NaN        NaN         NaN  1.562005   
          2689A         NaN        NaN        NaN         NaN       NaN   
          2691A         NaN        NaN        NaN  171.313395       NaN   
          2692A         NaN        NaN  71.747782         NaN       NaN   
          2693A         NaN        NaN        NaN   40.965761       NaN   
          2694A         NaN        NaN        NaN   54.859959       NaN   
          2695A  135.494878        NaN  47.994043         NaN       NaN   
          2696A         NaN        NaN  49.493930  104.385791  1.918970   
          2697A         NaN        NaN        NaN   81.714389       NaN   
          2698A  244.438238        NaN  58.382703  145.339615  2.734157   
          2699A         NaN        NaN  65.098090         NaN  2.973258   
          2700A         NaN        NaN  50.795986  190.893499  2.805873   
          2701A         NaN        NaN  43.519320   53.560539       NaN   
          2702A         NaN        NaN        NaN   81.700480       NaN   
          2703A         NaN  21.277004  59.691717         NaN  3.221161   
          2704A         NaN  26.426853  47.

In [19]:
pred.to_csv(os.path.join(input_root, "regression_main.csv"))